# PPTX Media Optimizer

Optimize PowerPoint files by compressing images, transcoding videos, and removing unused media.

**Features:**
- Image compression (JPEG/PNG) with transparency preservation
- Video transcoding to H.264/AAC (GPU accelerated when available)
- Audio optimization
- Unused media detection and removal

**Workflow:**
1. Upload your PPTX file
2. Run analysis to see optimization opportunities
3. Toggle which optimizations to apply
4. Execute and download optimized file

## 1. Setup & Dependencies

In [ ]:
#@title Install Dependencies & Detect GPU
import subprocess
import sys

# Install Python packages
print("Installing Python dependencies...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "python-pptx", "Pillow"])

# Check FFmpeg
print("\nChecking FFmpeg...")
try:
    result = subprocess.run(["ffmpeg", "-version"], capture_output=True, text=True)
    print(f"FFmpeg: {result.stdout.split(chr(10))[0]}")
except FileNotFoundError:
    print("FFmpeg not found, installing...")
    subprocess.run(["apt-get", "update", "-qq"])
    subprocess.run(["apt-get", "install", "-y", "-qq", "ffmpeg"])

# GPU Detection
print("\nDetecting GPU...")
GPU_AVAILABLE = False
NVENC_AVAILABLE = False

try:
    result = subprocess.run(["nvidia-smi", "--query-gpu=name,memory.total", "--format=csv,noheader"],
                          capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        GPU_AVAILABLE = True
        gpu_info = result.stdout.strip()
        print(f"GPU detected: {gpu_info}")
        
        # Check NVENC support
        result = subprocess.run(["ffmpeg", "-encoders"], capture_output=True, text=True)
        if "h264_nvenc" in result.stdout:
            NVENC_AVAILABLE = True
            print("NVENC encoder: Available")
        else:
            print("NVENC encoder: Not available (will use CPU)")
except Exception as e:
    print(f"No GPU detected: {e}")

if not GPU_AVAILABLE:
    print("Running in CPU-only mode")

print("\n" + "="*50)
print("Setup complete!")

## 2. Core Utilities

In [ ]:
#@title Core PPTX Utilities
import zipfile
import tempfile
import shutil
import os
import xml.etree.ElementTree as ET
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict, Set, Optional
from enum import Enum

class MediaType(Enum):
    IMAGE = "image"
    VIDEO = "video"
    AUDIO = "audio"
    VECTOR = "vector"
    UNKNOWN = "unknown"

@dataclass
class MediaFile:
    path: Path
    name: str
    size: int
    media_type: MediaType
    extension: str
    is_referenced: bool = True
    optimize: bool = True
    # Image-specific
    width: Optional[int] = None
    height: Optional[int] = None
    # Video/Audio-specific
    duration: Optional[float] = None
    codec: Optional[str] = None
    bitrate: Optional[int] = None
    # Estimated savings
    estimated_savings: int = 0

@dataclass
class AnalysisResult:
    original_size: int
    media_files: List[MediaFile] = field(default_factory=list)
    unused_files: List[MediaFile] = field(default_factory=list)
    total_media_size: int = 0
    potential_savings: int = 0

# File extension mappings
IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".tif"}
VECTOR_EXTENSIONS = {".emf", ".wmf", ".svg"}
VIDEO_EXTENSIONS = {".mp4", ".avi", ".mov", ".wmv", ".m4v", ".mkv", ".webm"}
AUDIO_EXTENSIONS = {".mp3", ".wav", ".m4a", ".wma", ".aac", ".ogg"}
SKIP_EXTENSIONS = {".gif"}  # GIF animation preservation is complex

def get_media_type(ext: str) -> MediaType:
    ext = ext.lower()
    if ext in IMAGE_EXTENSIONS:
        return MediaType.IMAGE
    if ext in VECTOR_EXTENSIONS:
        return MediaType.VECTOR
    if ext in VIDEO_EXTENSIONS:
        return MediaType.VIDEO
    if ext in AUDIO_EXTENSIONS:
        return MediaType.AUDIO
    return MediaType.UNKNOWN

def extract_pptx(pptx_path: str, extract_dir: str) -> bool:
    """Extract PPTX to directory. Returns True if valid ZIP."""
    try:
        with zipfile.ZipFile(pptx_path, 'r') as zf:
            zf.extractall(extract_dir)
        return True
    except zipfile.BadZipFile:
        print(f"Error: {pptx_path} is not a valid ZIP/PPTX file")
        return False

def repackage_pptx(source_dir: str, output_path: str) -> bool:
    """Repackage directory into PPTX. Preserves ZIP structure."""
    try:
        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zf:
            for root, dirs, files in os.walk(source_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    arc_name = os.path.relpath(file_path, source_dir)
                    zf.write(file_path, arc_name)
        return True
    except Exception as e:
        print(f"Error repackaging: {e}")
        return False

def get_referenced_media(extract_dir: str) -> Set[str]:
    """Parse all .rels files to find referenced media."""
    referenced = set()
    rels_dirs = [
        "ppt/slides/_rels",
        "ppt/slideLayouts/_rels",
        "ppt/slideMasters/_rels",
        "ppt/notesSlides/_rels",
        "ppt/_rels"
    ]
    
    ns = {"r": "http://schemas.openxmlformats.org/package/2006/relationships"}
    
    for rels_dir in rels_dirs:
        rels_path = Path(extract_dir) / rels_dir
        if not rels_path.exists():
            continue
        
        for rels_file in rels_path.glob("*.rels"):
            try:
                tree = ET.parse(rels_file)
                root = tree.getroot()
                
                for rel in root.findall(".//r:Relationship", ns):
                    target = rel.get("Target", "")
                    if "media/" in target:
                        # Extract just the filename
                        media_name = target.split("media/")[-1]
                        referenced.add(media_name)
            except ET.ParseError:
                continue
    
    return referenced

def format_size(size_bytes: int) -> str:
    """Format bytes to human readable string."""
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size_bytes < 1024:
            return f"{size_bytes:.1f} {unit}"
        size_bytes /= 1024
    return f"{size_bytes:.1f} TB"

print("Core utilities loaded.")

## 3. Analysis Engine

In [ ]:
#@title Analysis Functions
from PIL import Image
import json

def get_image_info(file_path: Path) -> tuple:
    """Get image dimensions."""
    try:
        with Image.open(file_path) as img:
            return img.width, img.height
    except Exception:
        return None, None

def get_media_info_ffprobe(file_path: Path) -> dict:
    """Get video/audio info using ffprobe."""
    try:
        cmd = [
            "ffprobe", "-v", "quiet",
            "-print_format", "json",
            "-show_format", "-show_streams",
            str(file_path)
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        if result.returncode == 0:
            return json.loads(result.stdout)
    except Exception:
        pass
    return {}

def estimate_image_savings(media: MediaFile, quality: int, max_width: int) -> int:
    """Estimate compression savings for an image."""
    if media.extension.lower() in VECTOR_EXTENSIONS or media.extension.lower() in SKIP_EXTENSIONS:
        return 0
    
    savings = 0
    # Estimate based on current size and target quality
    if media.width and media.width > max_width:
        # Resizing typically saves 50-70%
        scale_factor = max_width / media.width
        savings = int(media.size * (1 - scale_factor * scale_factor) * 0.7)
    else:
        # Compression typically saves 20-50% depending on quality
        compression_ratio = (100 - quality) / 100 * 0.5
        savings = int(media.size * compression_ratio)
    
    return max(0, savings)

def estimate_video_savings(media: MediaFile, target_bitrate: str) -> int:
    """Estimate transcoding savings for video."""
    if not media.duration or not media.bitrate:
        # Rough estimate: assume 30% savings
        return int(media.size * 0.3)
    
    # Parse target bitrate (e.g., "2M" -> 2000000)
    target_bps = int(float(target_bitrate.rstrip('MmKk')) * (1000000 if 'M' in target_bitrate.upper() else 1000))
    
    if media.bitrate > target_bps:
        new_size = (target_bps / media.bitrate) * media.size
        return int(media.size - new_size)
    return 0

def analyze_pptx(pptx_path: str, image_quality: int = 85, max_width: int = 1920, 
                 video_bitrate: str = "2M") -> AnalysisResult:
    """Analyze PPTX and return detailed media information."""
    result = AnalysisResult(original_size=os.path.getsize(pptx_path))
    
    # Create temp directory for extraction
    temp_dir = tempfile.mkdtemp(prefix="pptx_analysis_")
    
    try:
        if not extract_pptx(pptx_path, temp_dir):
            return result
        
        media_dir = Path(temp_dir) / "ppt" / "media"
        if not media_dir.exists():
            print("No media folder found in PPTX")
            return result
        
        # Get referenced media
        referenced = get_referenced_media(temp_dir)
        
        # Analyze each media file
        for media_file in media_dir.iterdir():
            if not media_file.is_file():
                continue
            
            ext = media_file.suffix.lower()
            media_type = get_media_type(ext)
            size = media_file.stat().st_size
            is_referenced = media_file.name in referenced
            
            media = MediaFile(
                path=media_file,
                name=media_file.name,
                size=size,
                media_type=media_type,
                extension=ext,
                is_referenced=is_referenced,
                optimize=is_referenced and media_type != MediaType.VECTOR
            )
            
            # Get detailed info based on type
            if media_type == MediaType.IMAGE:
                media.width, media.height = get_image_info(media_file)
                if ext not in SKIP_EXTENSIONS:
                    media.estimated_savings = estimate_image_savings(media, image_quality, max_width)
            
            elif media_type in (MediaType.VIDEO, MediaType.AUDIO):
                info = get_media_info_ffprobe(media_file)
                if info:
                    fmt = info.get("format", {})
                    media.duration = float(fmt.get("duration", 0))
                    media.bitrate = int(fmt.get("bit_rate", 0))
                    
                    for stream in info.get("streams", []):
                        if stream.get("codec_type") == "video":
                            media.codec = stream.get("codec_name")
                            media.width = stream.get("width")
                            media.height = stream.get("height")
                            break
                        elif stream.get("codec_type") == "audio" and not media.codec:
                            media.codec = stream.get("codec_name")
                
                if media_type == MediaType.VIDEO:
                    media.estimated_savings = estimate_video_savings(media, video_bitrate)
                else:
                    # Audio: estimate 20% savings
                    media.estimated_savings = int(size * 0.2)
            
            result.total_media_size += size
            
            if is_referenced:
                result.media_files.append(media)
                if media.optimize:
                    result.potential_savings += media.estimated_savings
            else:
                result.unused_files.append(media)
                result.potential_savings += size  # Full size saved by removal
    
    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)
    
    return result

def print_analysis(result: AnalysisResult):
    """Print analysis results in a readable format."""
    print("=" * 60)
    print("PPTX MEDIA ANALYSIS")
    print("=" * 60)
    print(f"\nOriginal file size: {format_size(result.original_size)}")
    print(f"Total media size: {format_size(result.total_media_size)}")
    print(f"Potential savings: {format_size(result.potential_savings)}")
    
    # Unused files
    if result.unused_files:
        print(f"\n{'='*60}")
        print(f"UNUSED MEDIA FILES ({len(result.unused_files)} files)")
        print("These files are not referenced and can be safely removed:")
        print("-" * 60)
        for media in result.unused_files:
            print(f"  [{media.media_type.value:6}] {media.name:30} {format_size(media.size):>10}")
        unused_total = sum(m.size for m in result.unused_files)
        print(f"  {'Total:':37} {format_size(unused_total):>10}")
    
    # Images
    images = [m for m in result.media_files if m.media_type == MediaType.IMAGE]
    if images:
        print(f"\n{'='*60}")
        print(f"IMAGES ({len(images)} files)")
        print("-" * 60)
        for media in images:
            dims = f"{media.width}x{media.height}" if media.width else "unknown"
            skip = " [SKIP]" if media.extension in SKIP_EXTENSIONS or media.extension in VECTOR_EXTENSIONS else ""
            savings = format_size(media.estimated_savings) if media.estimated_savings > 0 else "-"
            print(f"  {media.name:30} {dims:>12} {format_size(media.size):>10} | ~{savings}{skip}")
    
    # Vectors
    vectors = [m for m in result.media_files if m.media_type == MediaType.VECTOR]
    if vectors:
        print(f"\n{'='*60}")
        print(f"VECTOR IMAGES ({len(vectors)} files) - Cannot be optimized")
        print("-" * 60)
        for media in vectors:
            print(f"  {media.name:30} {format_size(media.size):>10}")
    
    # Videos
    videos = [m for m in result.media_files if m.media_type == MediaType.VIDEO]
    if videos:
        print(f"\n{'='*60}")
        print(f"VIDEOS ({len(videos)} files)")
        print("-" * 60)
        for media in videos:
            dims = f"{media.width}x{media.height}" if media.width else "unknown"
            dur = f"{media.duration:.1f}s" if media.duration else "unknown"
            codec = media.codec or "unknown"
            bitrate = f"{media.bitrate/1000000:.1f}Mbps" if media.bitrate else "unknown"
            savings = format_size(media.estimated_savings) if media.estimated_savings > 0 else "-"
            print(f"  {media.name:25} {dims:>10} {dur:>8} {codec:>8} {bitrate:>10} {format_size(media.size):>10} | ~{savings}")
    
    # Audio
    audio = [m for m in result.media_files if m.media_type == MediaType.AUDIO]
    if audio:
        print(f"\n{'='*60}")
        print(f"AUDIO ({len(audio)} files)")
        print("-" * 60)
        for media in audio:
            dur = f"{media.duration:.1f}s" if media.duration else "unknown"
            codec = media.codec or "unknown"
            savings = format_size(media.estimated_savings) if media.estimated_savings > 0 else "-"
            print(f"  {media.name:30} {dur:>10} {codec:>8} {format_size(media.size):>10} | ~{savings}")
    
    print("\n" + "=" * 60)

print("Analysis engine loaded.")

## 4. Image Optimizer

In [ ]:
#@title Image Optimization Functions
from PIL import Image
import io

def optimize_image(file_path: Path, quality: int = 85, max_width: int = 1920) -> tuple:
    """
    Optimize a single image file.
    Returns (success, original_size, new_size).
    """
    ext = file_path.suffix.lower()
    
    # Skip non-optimizable formats
    if ext in VECTOR_EXTENSIONS or ext in SKIP_EXTENSIONS:
        return False, 0, 0
    
    original_size = file_path.stat().st_size
    
    try:
        with Image.open(file_path) as img:
            original_mode = img.mode
            original_format = img.format
            
            # Resize if needed
            if img.width > max_width:
                ratio = max_width / img.width
                new_height = int(img.height * ratio)
                img = img.resize((max_width, new_height), Image.Resampling.LANCZOS)
            
            # Determine output format and settings
            if ext in (".jpg", ".jpeg"):
                # Convert RGBA to RGB for JPEG
                if img.mode == "RGBA":
                    background = Image.new("RGB", img.size, (255, 255, 255))
                    background.paste(img, mask=img.split()[3])
                    img = background
                img.save(file_path, "JPEG", quality=quality, optimize=True)
            
            elif ext == ".png":
                # Preserve transparency for PNG
                if img.mode == "RGBA":
                    img.save(file_path, "PNG", optimize=True)
                else:
                    # For non-transparent PNG, can use more aggressive compression
                    img.save(file_path, "PNG", optimize=True)
            
            elif ext in (".bmp", ".tiff", ".tif"):
                # Convert to PNG for better compression (keeping same extension would be invalid)
                # Instead, just optimize in place
                img.save(file_path, original_format, optimize=True)
            
            else:
                # Unknown format, try to save as-is
                img.save(file_path, optimize=True)
        
        new_size = file_path.stat().st_size
        return True, original_size, new_size
    
    except Exception as e:
        print(f"  Warning: Failed to optimize {file_path.name}: {e}")
        return False, original_size, original_size

def optimize_images(media_dir: Path, files_to_optimize: List[MediaFile], 
                   quality: int = 85, max_width: int = 1920) -> dict:
    """
    Optimize multiple image files.
    Returns dict with stats.
    """
    stats = {
        "processed": 0,
        "skipped": 0,
        "original_total": 0,
        "optimized_total": 0
    }
    
    images = [f for f in files_to_optimize if f.media_type == MediaType.IMAGE and f.optimize]
    
    for media in images:
        file_path = media_dir / media.name
        if not file_path.exists():
            stats["skipped"] += 1
            continue
        
        print(f"  Processing {media.name}...", end=" ")
        success, orig_size, new_size = optimize_image(file_path, quality, max_width)
        
        if success:
            savings = orig_size - new_size
            pct = (savings / orig_size * 100) if orig_size > 0 else 0
            print(f"{format_size(orig_size)} -> {format_size(new_size)} ({pct:.1f}% saved)")
            stats["processed"] += 1
            stats["original_total"] += orig_size
            stats["optimized_total"] += new_size
        else:
            print("skipped")
            stats["skipped"] += 1
    
    return stats

print("Image optimizer loaded.")

## 5. Video/Audio Optimizer

In [ ]:
#@title Video/Audio Optimization Functions

def transcode_video(input_path: Path, output_path: Path, bitrate: str = "2M", 
                   use_gpu: bool = True) -> tuple:
    """
    Transcode video to H.264/AAC MP4.
    Returns (success, original_size, new_size).
    """
    original_size = input_path.stat().st_size
    
    # Build FFmpeg command
    cmd = ["ffmpeg", "-y", "-hide_banner", "-loglevel", "warning"]
    
    if use_gpu and NVENC_AVAILABLE:
        # GPU-accelerated encoding
        cmd.extend([
            "-hwaccel", "cuda",
            "-i", str(input_path),
            "-c:v", "h264_nvenc",
            "-preset", "fast",
            "-b:v", bitrate,
            "-c:a", "aac",
            "-b:a", "128k",
            str(output_path)
        ])
    else:
        # CPU encoding
        cmd.extend([
            "-i", str(input_path),
            "-c:v", "libx264",
            "-preset", "medium",
            "-crf", "23",
            "-b:v", bitrate,
            "-maxrate", str(int(float(bitrate.rstrip('MmKk')) * 1.5)) + "M",
            "-bufsize", bitrate,
            "-c:a", "aac",
            "-b:a", "128k",
            str(output_path)
        ])
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=600)
        
        if result.returncode == 0 and output_path.exists():
            new_size = output_path.stat().st_size
            return True, original_size, new_size
        else:
            print(f"    FFmpeg error: {result.stderr[:200]}")
            return False, original_size, original_size
    
    except subprocess.TimeoutExpired:
        print("    Timeout during transcoding")
        return False, original_size, original_size
    except Exception as e:
        print(f"    Error: {e}")
        return False, original_size, original_size

def transcode_audio(input_path: Path, output_path: Path, bitrate: str = "128k") -> tuple:
    """
    Transcode audio to AAC.
    Returns (success, original_size, new_size).
    """
    original_size = input_path.stat().st_size
    
    cmd = [
        "ffmpeg", "-y", "-hide_banner", "-loglevel", "warning",
        "-i", str(input_path),
        "-c:a", "aac",
        "-b:a", bitrate,
        str(output_path)
    ]
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=120)
        
        if result.returncode == 0 and output_path.exists():
            new_size = output_path.stat().st_size
            return True, original_size, new_size
        else:
            return False, original_size, original_size
    except Exception as e:
        print(f"    Error: {e}")
        return False, original_size, original_size

def optimize_media(media_dir: Path, files_to_optimize: List[MediaFile],
                  video_bitrate: str = "2M", use_gpu: bool = True) -> dict:
    """
    Optimize video and audio files.
    Returns dict with stats.
    """
    stats = {
        "videos_processed": 0,
        "audio_processed": 0,
        "skipped": 0,
        "original_total": 0,
        "optimized_total": 0
    }
    
    videos = [f for f in files_to_optimize if f.media_type == MediaType.VIDEO and f.optimize]
    audio = [f for f in files_to_optimize if f.media_type == MediaType.AUDIO and f.optimize]
    
    encoder = "GPU (NVENC)" if use_gpu and NVENC_AVAILABLE else "CPU (libx264)"
    if videos:
        print(f"\nUsing {encoder} for video encoding")
    
    # Process videos
    for media in videos:
        input_path = media_dir / media.name
        if not input_path.exists():
            stats["skipped"] += 1
            continue
        
        # Output to temp file, then replace
        temp_output = input_path.with_suffix(".temp.mp4")
        
        print(f"  Transcoding {media.name}...", end=" ", flush=True)
        success, orig_size, new_size = transcode_video(input_path, temp_output, video_bitrate, use_gpu)
        
        if success:
            # Keep original extension to maintain XML references
            # But the content is now H.264 MP4
            input_path.unlink()
            temp_output.rename(input_path.with_suffix(".mp4"))
            
            # If original wasn't .mp4, we need to update the extension
            if input_path.suffix.lower() != ".mp4":
                # Note: This may break XML refs - ideally keep same extension
                # For safety, we'll rename to original extension but content is MP4
                new_path = input_path.with_suffix(".mp4")
                if input_path.suffix.lower() != ".mp4":
                    # Keep original name for XML compatibility
                    new_path.rename(input_path)
            
            savings = orig_size - new_size
            pct = (savings / orig_size * 100) if orig_size > 0 else 0
            print(f"{format_size(orig_size)} -> {format_size(new_size)} ({pct:.1f}% saved)")
            stats["videos_processed"] += 1
            stats["original_total"] += orig_size
            stats["optimized_total"] += new_size
        else:
            print("failed")
            if temp_output.exists():
                temp_output.unlink()
            stats["skipped"] += 1
    
    # Process audio
    for media in audio:
        input_path = media_dir / media.name
        if not input_path.exists():
            stats["skipped"] += 1
            continue
        
        temp_output = input_path.with_suffix(".temp.m4a")
        
        print(f"  Transcoding {media.name}...", end=" ", flush=True)
        success, orig_size, new_size = transcode_audio(input_path, temp_output)
        
        if success:
            input_path.unlink()
            # Keep original extension for XML compatibility
            temp_output.rename(input_path)
            
            savings = orig_size - new_size
            pct = (savings / orig_size * 100) if orig_size > 0 else 0
            print(f"{format_size(orig_size)} -> {format_size(new_size)} ({pct:.1f}% saved)")
            stats["audio_processed"] += 1
            stats["original_total"] += orig_size
            stats["optimized_total"] += new_size
        else:
            print("failed")
            if temp_output.exists():
                temp_output.unlink()
            stats["skipped"] += 1
    
    return stats

print("Video/Audio optimizer loaded.")

## 6. Unused Media Cleanup

In [ ]:
#@title Unused Media Cleanup Functions

def remove_unused_media(media_dir: Path, unused_files: List[MediaFile]) -> dict:
    """
    Remove unreferenced media files.
    Returns dict with stats.
    """
    stats = {
        "removed": 0,
        "bytes_freed": 0,
        "failed": 0
    }
    
    for media in unused_files:
        file_path = media_dir / media.name
        if not file_path.exists():
            continue
        
        try:
            size = file_path.stat().st_size
            file_path.unlink()
            print(f"  Removed {media.name} ({format_size(size)})")
            stats["removed"] += 1
            stats["bytes_freed"] += size
        except Exception as e:
            print(f"  Failed to remove {media.name}: {e}")
            stats["failed"] += 1
    
    return stats

print("Cleanup functions loaded.")

## 7. Main Workflow

In [ ]:
#@title Upload PPTX File
from google.colab import files
import os

print("Select your PPTX file to upload:")
uploaded = files.upload()

if uploaded:
    INPUT_FILE = list(uploaded.keys())[0]
    print(f"\nUploaded: {INPUT_FILE}")
    print(f"Size: {format_size(os.path.getsize(INPUT_FILE))}")
else:
    print("No file uploaded.")

In [ ]:
#@title Analyze PPTX
#@markdown Run this cell to analyze the uploaded PPTX file.

image_quality = 85 #@param {type:"slider", min:10, max:100, step:5}
max_image_width = 1920 #@param {type:"integer"}
video_bitrate = "2M" #@param {type:"string"}

print(f"Analyzing {INPUT_FILE}...\n")
analysis = analyze_pptx(INPUT_FILE, image_quality, max_image_width, video_bitrate)
print_analysis(analysis)

# Store for next step
ANALYSIS_RESULT = analysis

In [ ]:
#@title Configure Optimizations
#@markdown Toggle which optimizations to apply:

optimize_images_flag = True #@param {type:"boolean"}
optimize_videos_flag = True #@param {type:"boolean"}
optimize_audio_flag = True #@param {type:"boolean"}
remove_unused_flag = True #@param {type:"boolean"}
use_gpu_flag = True #@param {type:"boolean"}

#@markdown ---
#@markdown **Image Settings:**
image_quality_final = 85 #@param {type:"slider", min:10, max:100, step:5}
max_image_width_final = 1920 #@param {type:"integer"}

#@markdown ---
#@markdown **Video Settings:**
video_bitrate_final = "2M" #@param {type:"string"}

print("Configuration:")
print(f"  Optimize images: {optimize_images_flag}")
print(f"  Optimize videos: {optimize_videos_flag}")
print(f"  Optimize audio: {optimize_audio_flag}")
print(f"  Remove unused: {remove_unused_flag}")
print(f"  Use GPU: {use_gpu_flag} (Available: {GPU_AVAILABLE}, NVENC: {NVENC_AVAILABLE})")
print(f"  Image quality: {image_quality_final}")
print(f"  Max image width: {max_image_width_final}px")
print(f"  Video bitrate: {video_bitrate_final}")

In [ ]:
#@title Execute Optimization
#@markdown Run this cell to apply the configured optimizations.

import time

start_time = time.time()
original_size = os.path.getsize(INPUT_FILE)

# Create working directory
work_dir = tempfile.mkdtemp(prefix="pptx_optimize_")
output_file = INPUT_FILE.replace(".pptx", "_optimized.pptx")

print(f"Extracting {INPUT_FILE}...")
if not extract_pptx(INPUT_FILE, work_dir):
    print("ERROR: Failed to extract PPTX")
else:
    media_dir = Path(work_dir) / "ppt" / "media"
    
    total_stats = {
        "images": {},
        "videos": {},
        "unused": {}
    }
    
    # Remove unused media first
    if remove_unused_flag and ANALYSIS_RESULT.unused_files:
        print(f"\n{'='*50}")
        print("REMOVING UNUSED MEDIA")
        print("-" * 50)
        total_stats["unused"] = remove_unused_media(media_dir, ANALYSIS_RESULT.unused_files)
    
    # Optimize images
    if optimize_images_flag:
        images = [f for f in ANALYSIS_RESULT.media_files if f.media_type == MediaType.IMAGE]
        if images:
            print(f"\n{'='*50}")
            print("OPTIMIZING IMAGES")
            print("-" * 50)
            total_stats["images"] = optimize_images(
                media_dir, images, image_quality_final, max_image_width_final
            )
    
    # Optimize videos and audio
    if optimize_videos_flag or optimize_audio_flag:
        files_to_process = []
        if optimize_videos_flag:
            files_to_process.extend([f for f in ANALYSIS_RESULT.media_files if f.media_type == MediaType.VIDEO])
        if optimize_audio_flag:
            files_to_process.extend([f for f in ANALYSIS_RESULT.media_files if f.media_type == MediaType.AUDIO])
        
        if files_to_process:
            print(f"\n{'='*50}")
            print("OPTIMIZING VIDEO/AUDIO")
            print("-" * 50)
            total_stats["videos"] = optimize_media(
                media_dir, files_to_process, video_bitrate_final, use_gpu_flag
            )
    
    # Repackage
    print(f"\n{'='*50}")
    print("REPACKAGING PPTX")
    print("-" * 50)
    
    if repackage_pptx(work_dir, output_file):
        new_size = os.path.getsize(output_file)
        elapsed = time.time() - start_time
        
        print(f"\n{'='*60}")
        print("OPTIMIZATION COMPLETE")
        print("=" * 60)
        print(f"\nOriginal size:  {format_size(original_size)}")
        print(f"Optimized size: {format_size(new_size)}")
        print(f"Savings:        {format_size(original_size - new_size)} ({(original_size - new_size) / original_size * 100:.1f}%)")
        print(f"Time elapsed:   {elapsed:.1f} seconds")
        print(f"\nOutput file: {output_file}")
        
        # Summary
        if total_stats["unused"]:
            print(f"\nUnused media removed: {total_stats['unused'].get('removed', 0)} files ({format_size(total_stats['unused'].get('bytes_freed', 0))})")
        if total_stats["images"]:
            print(f"Images optimized: {total_stats['images'].get('processed', 0)} files")
        if total_stats["videos"]:
            print(f"Videos optimized: {total_stats['videos'].get('videos_processed', 0)} files")
            print(f"Audio optimized: {total_stats['videos'].get('audio_processed', 0)} files")
    else:
        print("ERROR: Failed to repackage PPTX")
    
    # Cleanup
    shutil.rmtree(work_dir, ignore_errors=True)

OUTPUT_FILE = output_file

In [ ]:
#@title Download Optimized File
from google.colab import files

if os.path.exists(OUTPUT_FILE):
    print(f"Downloading {OUTPUT_FILE}...")
    files.download(OUTPUT_FILE)
else:
    print("No optimized file found. Run the optimization first.")